# Introduction

This project is about analyzing app data from Google Play and Apple App stores. We'll pretend we're data analysts working for a company building Android and iOS apps in English. Our company only build apps that are free, and revenue comes from in-app ads. 

We'd like to profile the most profitable apps on the Google Play and Apple App stores Going through the data will help our developers understand what types of apps users gravitate towards.

The data for [Google Play][1] and [Apple App Store][2] can be downloaded at Kaggle.

[1]:https://www.kaggle.com/lava18/google-play-store-apps
[2]:https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps

## Opening and Exploring the Data

First we open the files and give them corresponding variable names. The header column is separated from the data for quick access. Here we will use the `explore_data()` function for exploration. It prints the rows in the list so they're readable, and finds the number of rows and columns if `rows_and_columns` is `True`. It assumes the input dataset doesn't have a header row.

The first few rows of each data set are printed along with the number of rows and columns. We also try and identify some columns that could help with our analysis.

In [66]:
from csv import reader

#Open .csv files
file1 = open("AppleStore.csv", encoding='utf8')
file2 = open("googleplaystore.csv", encoding='utf8')

apple_file = reader(file1)
apple_apps_data = list(apple_file) #lists of list

google_file = reader(file2)
google_apps_data = list(google_file)

In [67]:
#Separating the header from the data set 
apple_head = apple_apps_data[0]
apple_data = apple_apps_data[1:]

google_head = google_apps_data[0]
google_data = google_apps_data[1:]

def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    
    for row in dataset_slice:
        print(row)
        print('\n') # new empty line for separation
        
    if rows_and_columns:
        print("Number of rows:", len(dataset))
        print("Number of columns:", len(dataset[0]))

#First few rows
print("Apple Rows")
explore_data(apple_data, 1, 3, True)
print('\n')
print("Google Rows")
explore_data(google_data, 1, 3, True)

Apple Rows
['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns: 16


Google Rows
['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


In [68]:
#Exploring Columns
print(apple_head, '\n')
print(google_head, '\n')

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 



The columns we could use need to be related to price (we develop free apps only) and the user ratings for the app. They're detailed in two tables here:

| Google Column Name | Description |
|:-----------:|:------------:|
| 'Rating' | User rating of the app |
| 'Installs' | Number of downloads |
| 'Price' | Price of the App |
| 'Type' | Whether an app is paid or free |

| Apple Column Name | Description |
|:---------:|:---------:|
| 'user_rating' | Average user rating (for all version) |
| 'user_rating_ver' | Average user rating (for current version) |
| 'Price' | Price of the app |

## Wrong Data

The discussion section for the Google Play Store data set describes an error for row 10472 (data set without the header). Printing row 10472, the header, and another row show the rating for row 10472 has a rating of 19, which is incorrect, since the maximum rating is 5. Therefore we'll delete this row.

In [69]:
print(google_head, '\n') #header
print(google_data[10472], '\n') # incorrect
print(google_data[10473]) #correct

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up'] 

['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


In [70]:

del google_data[10472] # Running this more than once will delete more data.

## Duplicate Apps
### Part One

From the discussion section for the Google Play Store data, duplicate entries for the same applications have been found. An example is Instagram:

In [71]:
for app in google_data:
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


Using a `for` loop we see that there's 1,181 duplicate apps. In this case, getting rid of the duplicate app data will make our analysis more accurate. The duplicates won't be removed randomly. Take the Instagram app duplicates. All the data in each row is the same except for the 4th entry, which the number of user reviews. The different amount of user reviews suggests the data was taken at different times. It seems the higher the number of reviews, the more recent the data is. As such, we will keep the row with the highest amount of user reviews.

In [72]:
duplicate = []
unique = []

for app in google_data:
    app_name = app[0]
    if app_name in unique:
        duplicate.append(app_name)
    else:
        unique.append(app_name)

print('Number of duplicate apps:', len(duplicate))
print('\n')
print('Examples of duplicate apps:', duplicate[:3])

Number of duplicate apps: 1181


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business']


### Part Two

After removing the duplicate Google Play Store apps we should be left with 9659 unique apps.

In [73]:
print('Length after duplicates:', len(google_data) - 1181)

Length after duplicates: 9659


To remove the duplicates, we'll create a dictionary where each key is a unqiue app name and the corresponding dictionary value is the highest number of reviews for that particular app. 

In [74]:
reviews_max = {} #empty dictionary

for app in google_data:
    name = app[0] #app name
    n_reviews = float(app[3]) #number of reviews for the app
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews

    elif name not in reviews_max:
        reviews_max[name] = n_reviews

len(reviews_max) # is 9659 as expected

9659

Here we use the `reviews_max`dictionary to remove the duplicate rows. The list `android_clean` will hold a list of lists of our cleaned data while the `already_added` list helps us keep track of already added apps.

In [75]:
android_clean = [] #list for new cleaned data set
already_added = [] #app names

for app in google_data:
    name = app[0]
    n_reviews = float(app[3])
    
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(app)
        already_added.append(name)

explore_data(android_clean,0,3,True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


## Removing Non-English Apps
### Part One

Exploring the data sets long enough, we find that some apps aren't in English:

In [76]:
#Non-English Apps
print(apple_data[813][1])
print(apple_data[6731][1])
print('\n')
print(android_clean[4412][0])
print(android_clean[7940][0])

爱奇艺PPS -《欢乐颂2》电视剧热播
【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜


中国語 AQリスニング
لعبة تقدر تربح DZ


Since our company only develops English apps, we'll remove the non-English ones. One way to do this is looking at each app name and find the characters, letters, and symbols that don't belong to the English alphabet.

The `ord()` function gives an ASCII demical number corresponding to the input character. For example, the decimal number for `b` is `98`. In ASCII, the characters for English are in the range 0 to 127, so any numbers greater than 127 don't belong to the set of English characters. 

Using this information, we'll build a function that will iterate through each indivdual character in the app names and check if the number is in the range 0 to 127.

In [77]:
#function that iterates over input string

def English(string):
    
    for character in string:
        if ord(character) > 127:
            return False
    
    return True

print(English('Instagram'))
print(English('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
False


Certain English app names are identified as `False` because they use characters like emojis and other symbols (™) aren't in the ASCII range.

In [78]:
print(English('Docs To Go™ Free Office Suite'))
print(English('Instachat 😜'))
print('\n')
print(ord('™'))
print(ord('😜'))

False
False


8482
128540


Using the current `English` function, we'll lose important data since many English apps will be incorrectly labeled. To reduce this impact, we'll remove an app only if the name has more than 3 non-ASCII characters.

In [79]:
def English(string):
    non_ascii = 0
    
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
    
    if non_ascii > 3:
        return False
    else:
        return True

In [80]:
print(English('Docs To Go™ Free Office Suite'))
print(English('Instachat 😜'))
print(English('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
True
False


In [81]:
#Filtering the data sets with English function

google_eng = []
apple_eng = []

for app in android_clean:
    name = app[0]
    
    if English(name) == True:
        google_eng.append(app)

for app in apple_data:
    name = app[1]
    
    if English(name) == True:
        apple_eng.append(app)

#Exploring to see how many remaining rows are left

explore_data(google_eng,0,3,True)
print('\n')
explore_data(apple_eng, 0,3,True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 

Now we have 9,614 Google Play apps and 6,183 Apple apps, all in English.

## Isolating the Free Apps

We'll need to isolate the free apps from the non-free ones in order to start our analysis. We'll loop through each data set to isolate the free ones.

In [82]:
google_free = []
apple_free = []

for app in google_eng:
    price = app[7]
    
    if price == '0':
        google_free.append(app)

for app in apple_eng:
    price = app[4]
    
    if price == '0.0':
        apple_free.append(app)

explore_data(google_free,0,3,True)
print('\n')
explore_data(apple_free,0,3,True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 8864
Number of columns: 13


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 

Our final data sets have 8864 Google Play apps and 3222 Apple apps.

## Most Common Apps by Genre
### Part One

As said in the introduction, we'd like to figure out the types of apps that attract the most users since our revenue is influenced by how many people are using our apps.

We'll use this validation strategy for app ideas:
    1. Build a Android version of the app and publish it on Google Play.
    2. If the app gets a large response, we'll develop further.
    3. After six months, if the app is profitable, we'll build an iOS version of the app and publish it on the App Store.

Since we want the app on both Google Play and the App Store, we want to find successful app profiles on both markets. To do this we'll generate frequency tables to find out what the most common genres in each market, referring to the `prime-genre` column in the Apple data set and the `Genres` and `Category` columns in the  Google data set. 

### Part Two

In [83]:
#Function to generate frequency tables that show percentage
def freq_table(dataset, index):
    freq_dict = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]

        if value in freq_dict:
            freq_dict[value] += 1
        else:
            freq_dict[value] = 1
    
    freq_percentage = {}
    
    for key in freq_dict:
        percent = (freq_dict[key] / total) * 100
        freq_percentage[key] = percent
        
    return freq_percentage
     
#Function that displays those percentages in descending order
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

### Part Three

In [84]:
display_table(apple_free, -5) #prime_genre

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


In the `prime_genre` column of the App Store data set, the most common genre is "Games" at 58.16% followed by "Entertainment" at 7.88%. Most of apps fall into the entertainment category ("Photo & Video", "Social Networking", etc.) while lifestyle apps ("Productivity", "Health & Fitness") fall in the middle. The general trend is Apple users mostly have entertainment apps. A successful app profile would likely be in the entertainment section; however, the frequency of the genre might not imply there's a large number of users. 

In [85]:
display_table(google_free, -4) #Genres

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

For the `Genres` column, the most common genres are "Tools" at 8.45%, "Entertainment" at 6.07%, and "Education" at 5.35%. Compared to the App Store, where most of the apps are designed for fun, the Google Play market is more balanced, with both productivity and fun apps. Near the bottom of the list, most of the multiple genre apps are labeled as "Education". The abundance of this label may be explained in the Category table below.

In [86]:
display_table(google_free, 1) #Category

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

The `Category` column gives a better breakdown of how the apps are grouped. We see the "Family" category is much larger than the others at 18.91%, followed by "Game" at 9.72% and "Tools" at 8.46%. Below we see that most of the apps in the Family section are for children and focused on education. Apps in the Family category with a focus on education seems to be the most successful profile.

![Family Category](http://localhost:8888/files/OneDrive/Projects/app-store-project/Google_Play_Family.png)

## Most Popular Apps by Genre 
### The App Store

We've seen the App Store is full of fun apps for entertainment, while the Google Play store is more balanced with both fun and entertainment. We'd like to find out which genres have the most users. In the Google Play data set, this information is in the `Installs` column, but there is no similar data in the App Store data set. Instead, we'll use the total number of user ratings, found in `rating_count_tot`, as a substitute. To start, we'll calulate the average number of user ratings per app genre.

In [87]:
prime_genre_freq = freq_table(apple_free, -5) #prime_genre

for genre in prime_genre_freq:
    total = 0 #sum of the number of user ratings
    len_genre = 0 #number of apps specific to each genre
    
    for app in apple_free:
        genre_app = app[-5]
        if genre_app == genre:
            user_ratings_tot = float(app[5])
            total += user_ratings_tot
            len_genre += 1
    
    user_ratings_avg = total / len_genre
    print(genre + ':', user_ratings_avg)

Social Networking: 71548.34905660378
Photo & Video: 28441.54375
Games: 22788.6696905016
Music: 57326.530303030304
Reference: 74942.11111111111
Health & Fitness: 23298.015384615384
Weather: 52279.892857142855
Utilities: 18684.456790123455
Travel: 28243.8
Shopping: 26919.690476190477
News: 21248.023255813954
Navigation: 86090.33333333333
Lifestyle: 16485.764705882353
Entertainment: 14029.830708661417
Food & Drink: 33333.92307692308
Sports: 23008.898550724636
Book: 39758.5
Finance: 31467.944444444445
Education: 7003.983050847458
Productivity: 21028.410714285714
Business: 7491.117647058823
Catalogs: 4004.0
Medical: 612.0


Looking at the results, we see that the `Navigation` genre has the highest average number of total user reviews. Waze and Google Maps skew the total with their review numbers being much higher than the others. This imbalance doesn't allow for an equal representation of the available applications.

In [88]:
for app in apple_free:
    name = app[1]
    total_ratings = float(app[5])
    if app[-5] == 'Navigation':
        print(name + ':', total_ratings) 

Waze - GPS Navigation, Maps & Real-time Traffic: 345046.0
Google Maps - Navigation & Transit: 154911.0
Geocaching®: 12811.0
CoPilot GPS – Car Navigation & Offline Maps: 3582.0
ImmobilienScout24: Real Estate Search in Germany: 187.0
Railway Route Search: 5.0


Given we found the most common genres in the App Store were about fun and entertainment, let's look through the "Social Networking" genre:

In [89]:
for app in apple_free:
    name = app[1]
    total_ratings = float(app[5])
    if app[-5] == 'Social Networking' and total_ratings > 0:
        print(name + ':', total_ratings) 

Facebook: 2974676.0
Pinterest: 1061624.0
Skype for iPhone: 373519.0
Messenger: 351466.0
Tumblr: 334293.0
WhatsApp Messenger: 287589.0
Kik: 260965.0
ooVoo – Free Video Call, Text and Voice: 177501.0
TextNow - Unlimited Text + Calls: 164963.0
Viber Messenger – Text & Call: 164249.0
Followers - Social Analytics For Instagram: 112778.0
MeetMe - Chat and Meet New People: 97072.0
We Heart It - Fashion, wallpapers, quotes, tattoos: 90414.0
InsTrack for Instagram - Analytics Plus More: 85535.0
Tango - Free Video Call, Voice and Chat: 75412.0
LinkedIn: 71856.0
Match™ - #1 Dating App.: 60659.0
Skype for iPad: 60163.0
POF - Best Dating App for Conversations: 52642.0
Timehop: 49510.0
Find My Family, Friends & iPhone - Life360 Locator: 43877.0
Whisper - Share, Express, Meet: 39819.0
Hangouts: 36404.0
LINE PLAY - Your Avatar World: 34677.0
WeChat: 34584.0
Badoo - Meet New People, Chat, Socialize.: 34428.0
Followers + for Instagram - Follower Analytics: 28633.0
GroupMe: 28260.0
Marco Polo Video Walki

Again, known popular app giants like Facebook and Tumblr dominate the results with review numbers over 100,000+. Smaller social apps struggle to compete in this market, each averaging around 300 to 500 reviews. Creating a social application in the `Social Networking` genre and getting it to be profitable will be a challenge. `Utilities` has on average 18,684 total reviews but the apps in this genre seem to be more balanced. 

In [90]:
for app in apple_free:
    name = app[1]
    total_ratings = float(app[5])
    if app[-5] == 'Utilities':
        print(name + ':', total_ratings) 

Google – Search made just for mobile: 479440.0
Flashlight Ⓞ: 130450.0
My Verizon: 126948.0
myAT&T: 108507.0
The Calculator - Free and Easy Calculating!: 99244.0
Speedtest by Ookla: 65016.0
Google Chrome – The Fast and Secure Web Browser: 55750.0
Alarm Clock HD Free - Digital Alarm Clock Display: 54496.0
Calculator Pro for iPad Free - Smart Calculator: 47803.0
Bitmoji - Your Personal Emoji: 37972.0
Calculator‰: 36879.0
Flashlight ！: 35769.0
Truecaller - Spam Identification & Block: 27791.0
Browser and File Manager for Documents: 27750.0
Flashlight for iPhone , iPod and iPad: 26697.0
RainbowKey - Color keyboard themes, fonts & GIF: 23063.0
Calculator HD Free: 14986.0
QR Reader for iPhone: 12683.0
Puffin Web Browser: 11677.0
Evolution Calculator - CP & XP - for Pokemon GO!: 9269.0
Gboard — a new keyboard from Google: 8873.0
Free QR Code Reader & Barcode Scanner for iPhone: 7343.0
GIF Keyboard: 5979.0
Alarm Clock for Me - Best Wake Up Music & Clock: 4959.0
Quick Scan - QR Code Reader: 4783

The `Utilities`genre looks a little more promising. Although the Google app holds the top spot, for most phones, most of these apps are already built-in. A lot of them are calculators, flashlights, or keyboard theme apps; they likely have extra features that make it worthwhile for users to download them.

One app idea could be a basic PDF reader and scanner. For downloaded PDFs, we could automatically create a table of contents based on the pages, and the user could modify and add bookmarks. There would also be pencil tools to edit. The scanner would need access to the camera, and the quality would only be as good as the phone camera itself. A small toolbar would have searching (for words and phrases), pencils, and the table of contents, along with a page by page scroll.

Since the App Store has mostly fun and social apps, an apps for tools could be succesful. It also fits in with the Google Play Store frequency table, where the popular genres and categories are mostly productivity applications.

Let's look at the most popular apps by genre for the Google Play Store